# B2. インデクサの監視とトラブルシューティング

## 概要

このノートブックでは、SharePoint インデクサの実行状況を監視し、問題を診断・解決します:

1. **実行履歴の確認**: 過去の実行結果の詳細
2. **エラー分析**: 失敗したアイテムとエラーメッセージ
3. **警告の確認**: 潜在的な問題の検出
4. **パフォーマンス分析**: 実行時間とスループット
5. **トラブルシューティング**: よくある問題と対処法
6. **部分再実行**: 失敗したアイテムの再処理

## 前提条件

- `B1_sp_indexer_setup.ipynb` が完了していること
- インデクサが作成されていること

## 1. 環境の初期化

In [ ]:
import os
import json
from pathlib import Path
from dotenv import load_dotenv
from typing import Dict, List, Optional
from datetime import datetime, timedelta
import time

from azure.core.credentials import AzureKeyCredential
from azure.search.documents.indexes import SearchIndexerClient
from azure.search.documents import SearchClient

# 環境変数の読み込み
env_path = Path("../.env")
if env_path.exists():
    load_dotenv(env_path)
    print("✅ .env ファイルを読み込みました")
else:
    print("⚠️ .env ファイルが見つかりません")

# 必須環境変数の確認
required_vars = ["AZURE_SEARCH_ENDPOINT", "AZURE_SEARCH_API_KEY"]
missing_vars = [var for var in required_vars if not os.getenv(var)]
if missing_vars:
    raise ValueError(f"環境変数が設定されていません: {', '.join(missing_vars)}")

print("\n✅ 環境変数の確認完了")

## 2. 設定とクライアントの初期化

In [ ]:
# Azure AI Search 設定
search_endpoint = os.getenv("AZURE_SEARCH_ENDPOINT")
search_key = os.getenv("AZURE_SEARCH_API_KEY")
index_name = os.getenv("AZURE_SEARCH_INDEX_NAME", "sp-docs-indexer")

# インデクサ名
indexer_name = f"{index_name}-indexer"

# 認証情報
credential = AzureKeyCredential(search_key)

# インデクサクライアント
indexer_client = SearchIndexerClient(
    endpoint=search_endpoint,
    credential=credential
)

# Search クライアント
search_client = SearchClient(
    endpoint=search_endpoint,
    index_name=index_name,
    credential=credential
)

print(f"✅ クライアント作成完了")
print(f"   エンドポイント: {search_endpoint}")
print(f"   インデクサ名: {indexer_name}")

## 3. ヘルパー関数

In [ ]:
def format_datetime(dt) -> str:
    """日時をフォーマット"""
    if dt is None:
        return "N/A"
    if isinstance(dt, str):
        return dt
    return dt.strftime("%Y-%m-%d %H:%M:%S")


def calculate_duration(start_time, end_time) -> str:
    """実行時間を計算"""
    if not start_time or not end_time:
        return "N/A"
    
    try:
        if isinstance(start_time, str):
            start_time = datetime.fromisoformat(start_time.replace('Z', '+00:00'))
        if isinstance(end_time, str):
            end_time = datetime.fromisoformat(end_time.replace('Z', '+00:00'))
        
        duration = end_time - start_time
        total_seconds = int(duration.total_seconds())
        
        if total_seconds < 60:
            return f"{total_seconds}秒"
        elif total_seconds < 3600:
            minutes = total_seconds // 60
            seconds = total_seconds % 60
            return f"{minutes}分{seconds}秒"
        else:
            hours = total_seconds // 3600
            minutes = (total_seconds % 3600) // 60
            return f"{hours}時間{minutes}分"
    except Exception:
        return "N/A"


def get_status_icon(status: str) -> str:
    """ステータスに応じたアイコンを返す"""
    status_lower = status.lower() if status else ""
    
    if "success" in status_lower:
        return "✅"
    elif "progress" in status_lower or "running" in status_lower:
        return "⏳"
    elif "fail" in status_lower or "error" in status_lower:
        return "❌"
    elif "reset" in status_lower:
        return "🔄"
    else:
        return "ℹ️"


print("✅ ヘルパー関数を定義しました")

## 4. 現在のステータス確認

In [ ]:
print(f"📊 インデクサ '{indexer_name}' の現在のステータス\n")
print("="*80)

try:
    status = indexer_client.get_indexer_status(indexer_name)
    
    # 全体のステータス
    icon = get_status_icon(str(status.status))
    print(f"\n{icon} 全体ステータス: {status.status}")
    
    # 実行モード
    if hasattr(status, 'execution_info'):
        print(f"\n実行情報:")
        print(f"  モード: {status.execution_info.get('mode', 'N/A')}")
    
    # 最新の実行結果
    if status.last_result:
        last = status.last_result
        
        print(f"\n最新の実行結果:")
        print(f"  状態: {get_status_icon(str(last.status))} {last.status}")
        print(f"  開始時刻: {format_datetime(last.start_time)}")
        print(f"  終了時刻: {format_datetime(last.end_time)}")
        print(f"  実行時間: {calculate_duration(last.start_time, last.end_time)}")
        
        # 処理統計
        if hasattr(last, 'item_count') and last.item_count is not None:
            print(f"\n処理統計:")
            print(f"  処理アイテム数: {last.item_count}")
            
            if hasattr(last, 'failed_item_count') and last.failed_item_count is not None:
                print(f"  失敗アイテム数: {last.failed_item_count}")
                
                success_rate = 0
                if last.item_count > 0:
                    success_rate = ((last.item_count - last.failed_item_count) / last.item_count) * 100
                print(f"  成功率: {success_rate:.1f}%")
        
        # エラーメッセージ
        if last.error_message:
            print(f"\n❌ エラーメッセージ:")
            print(f"  {last.error_message}")
        
        # エラー詳細
        if last.errors:
            print(f"\n❌ エラー詳細 ({len(last.errors)} 件):")
            for i, error in enumerate(last.errors[:5], 1):  # 最大5件表示
                print(f"\n  {i}. キー: {error.key if hasattr(error, 'key') else 'N/A'}")
                print(f"     メッセージ: {error.error_message if hasattr(error, 'error_message') else error}")
                if hasattr(error, 'status_code'):
                    print(f"     ステータスコード: {error.status_code}")
            
            if len(last.errors) > 5:
                print(f"\n  ... 他 {len(last.errors) - 5} 件のエラー")
        
        # 警告
        if last.warnings:
            print(f"\n⚠️ 警告 ({len(last.warnings)} 件):")
            for i, warning in enumerate(last.warnings[:5], 1):  # 最大5件表示
                print(f"\n  {i}. キー: {warning.key if hasattr(warning, 'key') else 'N/A'}")
                print(f"     メッセージ: {warning.message if hasattr(warning, 'message') else warning}")
            
            if len(last.warnings) > 5:
                print(f"\n  ... 他 {len(last.warnings) - 5} 件の警告")
    
    else:
        print("\nℹ️ まだ実行結果がありません")
    
    print("\n" + "="*80)

except Exception as e:
    print(f"❌ ステータス取得エラー: {e}")

## 5. 実行履歴の確認

過去の実行履歴を確認します。

In [ ]:
print(f"📜 インデクサ '{indexer_name}' の実行履歴\n")
print("="*80)

try:
    status = indexer_client.get_indexer_status(indexer_name)
    
    if hasattr(status, 'execution_history') and status.execution_history:
        history = status.execution_history
        print(f"\n実行履歴: {len(history)} 件\n")
        
        for i, execution in enumerate(history[:10], 1):  # 最大10件表示
            print(f"\n{i}. {get_status_icon(str(execution.status))} {execution.status}")
            print(f"   開始: {format_datetime(execution.start_time)}")
            print(f"   終了: {format_datetime(execution.end_time)}")
            print(f"   時間: {calculate_duration(execution.start_time, execution.end_time)}")
            
            if hasattr(execution, 'item_count') and execution.item_count is not None:
                print(f"   処理: {execution.item_count} 件")
                
                if hasattr(execution, 'failed_item_count') and execution.failed_item_count is not None:
                    print(f"   失敗: {execution.failed_item_count} 件")
            
            if execution.error_message:
                print(f"   エラー: {execution.error_message[:100]}...")
        
        if len(history) > 10:
            print(f"\n... 他 {len(history) - 10} 件の実行履歴")
    else:
        print("\nℹ️ 実行履歴がありません")
    
    print("\n" + "="*80)

except Exception as e:
    print(f"❌ 履歴取得エラー: {e}")

## 6. インデックスの統計情報

In [ ]:
print(f"📊 インデックス '{index_name}' の統計情報\n")
print("="*80)

try:
    # ドキュメント総数
    results = search_client.search(search_text="*", top=0, include_total_count=True)
    total_docs = results.get_count()
    
    print(f"\n総ドキュメント数: {total_docs}")
    
    if total_docs > 0:
        # ファイルタイプ別の集計
        print(f"\nファイルタイプ別:")
        
        # 簡易的な集計（実際にはfacetを使うとより効率的）
        content_types = {}
        sample_results = search_client.search(
            search_text="*",
            top=1000,  # サンプルとして1000件取得
            select=["contentType"]
        )
        
        for doc in sample_results:
            ct = doc.get('contentType', 'unknown')
            content_types[ct] = content_types.get(ct, 0) + 1
        
        for ct, count in sorted(content_types.items(), key=lambda x: x[1], reverse=True):
            print(f"  {ct}: {count} 件")
        
        # 最近更新されたドキュメント
        print(f"\n最近更新されたドキュメント (上位5件):")
        recent_results = search_client.search(
            search_text="*",
            top=5,
            order_by=["lastModified desc"],
            select=["title", "lastModified", "contentType"]
        )
        
        for i, doc in enumerate(recent_results, 1):
            print(f"  {i}. {doc.get('title', 'N/A')}")
            print(f"     更新: {format_datetime(doc.get('lastModified'))}")
            print(f"     種類: {doc.get('contentType', 'N/A')}")
    
    print("\n" + "="*80)

except Exception as e:
    print(f"❌ 統計取得エラー: {e}")

## 7. トラブルシューティングガイド

よくある問題と対処法を確認します。

In [ ]:
print("🔧 トラブルシューティングガイド\n")
print("="*80)

troubleshooting_tips = [
    {
        "問題": "インデクサが失敗する",
        "原因": [
            "SharePoint への接続権限がない",
            "アプリ登録の認証情報が間違っている",
            "管理者の同意が完了していない"
        ],
        "対処法": [
            "1. Azure Portal でアプリ登録を確認",
            "2. Sites.Read.All 権限があるか確認",
            "3. 管理者の同意が有効か確認",
            "4. データソースの接続文字列を確認"
        ]
    },
    {
        "問題": "ドキュメントが取り込まれない",
        "原因": [
            "サイトパスが間違っている",
            "ライブラリ名が間違っている",
            "対象ファイルが除外されている"
        ],
        "対処法": [
            "1. SharePoint のサイトURLを確認",
            "2. ライブラリ名を確認（大文字小文字区別）",
            "3. データソースのクエリ設定を確認",
            "4. ファイル拡張子フィルタを確認"
        ]
    },
    {
        "問題": "特定のファイルが失敗する",
        "原因": [
            "ファイルサイズが大きすぎる",
            "サポートされていないファイル形式",
            "ファイルが破損している"
        ],
        "対処法": [
            "1. エラーログでファイル名を確認",
            "2. ファイルサイズ制限を確認（通常16MB）",
            "3. ファイルを手動でダウンロードして確認",
            "4. 問題のファイルを除外設定に追加"
        ]
    },
    {
        "問題": "実行が遅い",
        "原因": [
            "大量のドキュメントを処理している",
            "スキルセットの処理が重い",
            "SharePoint のスロットリング"
        ],
        "対処法": [
            "1. バッチサイズを調整",
            "2. スキルセットを最適化",
            "3. スケジュール間隔を調整",
            "4. パーティション数を増やす（Search サービス）"
        ]
    },
    {
        "問題": "ベクター化が動作しない",
        "原因": [
            "Azure OpenAI の設定が間違っている",
            "デプロイメント名が間違っている",
            "APIキーが無効"
        ],
        "対処法": [
            "1. Azure OpenAI エンドポイントを確認",
            "2. デプロイメント名を確認",
            "3. APIキーを再生成",
            "4. スキルセットの設定を確認"
        ]
    }
]

for i, tip in enumerate(troubleshooting_tips, 1):
    print(f"\n{i}. 問題: {tip['問題']}")
    print(f"\n   考えられる原因:")
    for cause in tip['原因']:
        print(f"   - {cause}")
    print(f"\n   対処法:")
    for solution in tip['対処法']:
        print(f"   {solution}")
    print()

print("="*80)

## 8. インデクサの手動実行

インデクサを手動で実行します。

In [ ]:
# 手動実行する場合は、下記のコメントを外してください

# print(f"🚀 インデクサ '{indexer_name}' を手動実行します...\n")
# 
# try:
#     indexer_client.run_indexer(indexer_name)
#     print("✅ インデクサの実行を開始しました")
#     print("\nℹ️ 実行状況は上記のセルで確認できます")
# except Exception as e:
#     print(f"❌ インデクサ実行エラー: {e}")

print("ℹ️ インデクサを手動実行する場合は、このセルのコメントを外してください")

## 9. インデクサのリセット

インデクサをリセットして、すべてのドキュメントを再処理します。

**警告**: これにより既存のインデックスがクリアされ、すべてのドキュメントが再取り込みされます。

In [ ]:
# リセットする場合は、下記のコメントを外してください

# print(f"🔄 インデクサ '{indexer_name}' をリセットします...\n")
# print("⚠️ 警告: すべてのドキュメントが再処理されます\n")
# 
# try:
#     indexer_client.reset_indexer(indexer_name)
#     print("✅ インデクサをリセットしました")
#     print("\nℹ️ 次回の実行時にすべてのドキュメントが再処理されます")
# except Exception as e:
#     print(f"❌ リセットエラー: {e}")

print("ℹ️ インデクサをリセットする場合は、このセルのコメントを外してください")

## 10. スケジュール設定の確認と変更

In [ ]:
print(f"⏰ インデクサ '{indexer_name}' のスケジュール設定\n")
print("="*80)

try:
    indexer = indexer_client.get_indexer(indexer_name)
    
    if indexer.schedule:
        schedule = indexer.schedule
        print(f"\n現在のスケジュール:")
        print(f"  間隔: {schedule.interval}")
        
        if hasattr(schedule, 'start_time') and schedule.start_time:
            print(f"  開始時刻: {format_datetime(schedule.start_time)}")
        
        # 間隔の説明
        interval_str = str(schedule.interval)
        if "PT1H" in interval_str:
            print(f"\n  ℹ️ 1時間ごとに実行されます")
        elif "PT30M" in interval_str:
            print(f"\n  ℹ️ 30分ごとに実行されます")
        elif "PT15M" in interval_str:
            print(f"\n  ℹ️ 15分ごとに実行されます")
        elif "P1D" in interval_str:
            print(f"\n  ℹ️ 1日ごとに実行されます")
    else:
        print("\nℹ️ スケジュールが設定されていません（手動実行のみ）")
    
    print("\n" + "="*80)
    print("\nℹ️ スケジュールを変更するには、インデクサの設定を更新してください")
    print("   例: PT15M (15分), PT30M (30分), PT1H (1時間), P1D (1日)")

except Exception as e:
    print(f"❌ スケジュール取得エラー: {e}")

## まとめ

このノートブックでは、SharePoint インデクサの監視とトラブルシューティングを行いました:

1. ✅ **現在のステータス確認**: 実行状態とエラーの確認
2. ✅ **実行履歴**: 過去の実行結果の分析
3. ✅ **統計情報**: インデックスの内容確認
4. ✅ **トラブルシューティング**: よくある問題と対処法
5. ✅ **手動実行とリセット**: インデクサの管理
6. ✅ **スケジュール確認**: 自動実行の設定

### 次のステップ

- **B3_query_and_acl_demo.ipynb**: 検索とACLフィルタのデモ
- エラーが多い場合は、データソースやスキルセットの設定を見直す
- 本番運用に向けたスケジュール調整

### 運用のベストプラクティス

1. **定期的な監視**: エラー率と処理時間を追跡
2. **アラート設定**: Azure Monitor で失敗時のアラートを設定
3. **スケジュール最適化**: データ更新頻度に応じた間隔設定
4. **コスト管理**: 不要な再処理を避ける

### 参考リソース

- [インデクサのトラブルシューティング](https://learn.microsoft.com/azure/search/search-indexer-troubleshooting)
- [インデクサの監視](https://learn.microsoft.com/azure/search/search-howto-monitor-indexers)
- [Azure Monitor との統合](https://learn.microsoft.com/azure/search/monitor-azure-cognitive-search)